In [1]:
import cv2
import numpy as np
import threading
import argparse
import sys
import os
import time
import csv
import tensorflow as tf
import mediapipe as mp
import numpy as np
from pathlib import Path

from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

from concurrent.futures import ThreadPoolExecutor


mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [2]:
n_time_steps = 5
model = tf.keras.models.load_model("model.h5")

In [3]:
mediapipe_pose_model_asset = "mediapipe_pose_landmarker_model/pose_landmarker_heavy.task"
#mediapipe_pose_model_asset = "mediapipe_pose_landmarker_model/pose_landmarker_full.task"
base_options = python.BaseOptions(model_asset_path=mediapipe_pose_model_asset)
options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.VIDEO,
        num_poses=1,
        min_pose_detection_confidence=0.5,
        min_pose_presence_confidence=0.5,
        min_tracking_confidence=0.5,
        output_segmentation_masks=False)


In [4]:
def make_landmark_timestep(results):
    c_lm = []
    def add_lanmark(index):
        landmark = results.pose_landmarks[0][index]
        c_lm.append(landmark.x)
        c_lm.append(landmark.y)
        c_lm.append(landmark.z)
        c_lm.append(landmark.visibility)
        
    add_lanmark(0)
    add_lanmark(11)
    add_lanmark(12)
    add_lanmark(13)
    add_lanmark(14)
    add_lanmark(15)
    add_lanmark(16)
    add_lanmark(23)
    add_lanmark(24)
    add_lanmark(25)
    add_lanmark(26)
    add_lanmark(27)
    add_lanmark(28)

    return c_lm


def draw_landmark_on_image(mpDraw, results, img):
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = img.shape
        print(id, lm)
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
    return img


def draw_class_on_image(label, img):
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10, 30)
    fontScale = 1
    fontColor = (0, 255, 0)
    thickness = 2
    lineType = 2
    cv2.putText(img, label,
                bottomLeftCornerOfText,
                font,
                fontScale,
                fontColor,
                thickness,
                lineType)
    return img

In [5]:
def detect(model, lm_list):
    global label
    lm_list = np.array(lm_list)
    lm_list = np.expand_dims(lm_list, axis=0)
    #print(lm_list.shape)
    results = model.predict(lm_list)
    #print(results)
    if results[0][0] > 0.5:
        label = "FALLING"
    else:
        label = "NOT FALL"
    return label

In [6]:
executor = ThreadPoolExecutor(max_workers=1)

def threaded_detect(model, lm_data):
    global label
    label = detect(model, lm_data)
    # Consider adding any post-detection logic here, if needed


In [15]:
file_name = "evaluation/test_nofall.mp4"
video = cv2.VideoCapture(file_name)
detector = vision.PoseLandmarker.create_from_options(options)
lm_list = []
label = "Please wait"
current_frame = 0
while True:
    success, frame = video.read()
    
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)
    
    # tren rpi, toc do toi da 5 fps
    results = detector.detect_for_video(mp_image, int(video.get(cv2.CAP_PROP_POS_MSEC)))
    current_frame += 1    
    if results.pose_landmarks:
        if current_frame % 6 == 0: # moi truong video, cu 6 frame thi lay keypoint day vao model, 
                                    # tren rpi k can phai check frame, day thang vao (vi co delay san)
            c_lm = make_landmark_timestep(results)
            lm_list.append(c_lm)
            if len(lm_list) >= n_time_steps:
                lm_data_to_predict = lm_list[-n_time_steps:]
                executor.submit(threaded_detect, model, lm_data_to_predict)
                lm_list.pop(0)
                #lm_list = []
           
        for pose_landmarks in results.pose_landmarks:
                # Draw the pose landmarks.
                pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                pose_landmarks_proto.landmark.extend([
                    landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y,
                                                    z=landmark.z) for landmark
                    in pose_landmarks
                ])
                mp_drawing.draw_landmarks(
                    frame,
                    pose_landmarks_proto,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing_styles.get_default_pose_landmarks_style())
    else:
        lm_list = []
        
    frame = draw_class_on_image(label, frame)
    cv2.imshow("Image", frame)
    if cv2.waitKey(1) == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

ValueError: Input timestamp must be monotonically increasing.